In [ ]:
import pandas as pd
import numpy as np
from imu import IMUNavigator
import os
from utils import lat_lon_to_ecef_with_height, time_difference

In [ ]:
oxts_data_fields = [
    "lat", "lon", "alt",
    "roll", "pitch", "yaw",
    "vn", "ve", "vf", "vl", "vu",
    "ax", "ay", "az",
    "af", "al", "au",
    "wx", "wy", "wz",
    "wf", "wl", "wu",
    "pos_accuracy", "vel_accuracy",
    "navstat", "numsats",
    "posmode", "velmode", "orimode"
]
timestamps_file = r"C:\javier\personal_projects\gps_ins_fusion\data\oxts\timestamps.txt"
oxts_folder = r"C:\javier\personal_projects\gps_ins_fusion\data\oxts\data"

# loading data
ts = pd.read_csv(timestamps_file, header=None)

data_list = []
for file_ in os.listdir(oxts_folder):
    if file_.endswith(".txt"):
        df = pd.read_csv(os.path.join(oxts_folder, file_), header=None, sep=" ")
        data_list.append(df.values.flatten())

oxts = pd.DataFrame(data_list)
oxts.columns = oxts_data_fields
oxts['timestamps'] = ts
oxts['x_ecef'], oxts['y_ecef'], oxts['z_ecef'] = lat_lon_to_ecef_with_height(oxts['lat'].values, oxts['lon'].values, oxts['alt'].values)
for field_ in ['x_ecef', 'y_ecef', 'z_ecef']:
    oxts[field_] = oxts[field_] - oxts[field_][0]

n_rows = oxts.shape[0]

# initialize IMUNavigator
initial_row = oxts.loc[0, :]
pos = initial_row[['x_ecef', 'y_ecef', 'z_ecef']].values
vel = np.hstack((initial_row[['vn', 've']].values, [0]))
imu_nav = IMUNavigator(pos, vel, initial_row['roll'], initial_row['pitch'], initial_row['yaw'])

filt_pos = np.zeros((n_rows, 3))
time_vec = np.zeros(n_rows)

for index, row in oxts.iterrows():
    if index > 0:
        # compute time difference
        dt = time_difference(oxts.loc[index - 1, 'timestamps'], row['timestamps'])

        # read IMU measures
        acc = row[['af', 'al', 'au']].values
        gyro = row[['wf', 'wl', 'wu']].values

        imu_nav.step(acc, gyro, dt)
        filt_pos[index, :] = imu_nav.pos
        time_vec[index] = time_vec[index - 1] + dt